In [33]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Embedding, LSTM, Dense # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
import keras



In [ ]:
data = pd.read_csv('extended_chat_data.csv', on_bad_lines='skip')

questions = data['Question'].values
answers = data['Answer'].values

data


In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

questions_seq = tokenizer.texts_to_sequences(questions)
answers_seq = tokenizer.texts_to_sequences(answers)

max_seq_len = max([len(seq) for seq in questions_seq])

questions_seq = pad_sequences(questions_seq, maxlen=max_seq_len, padding='post')
answers_seq = pad_sequences(answers_seq, maxlen=max_seq_len, padding='post')

x_train, x_test, y_train, y_test = train_test_split(questions_seq, answers_seq, test_size=0.2)


In [ ]:
print(y_train.shape)
print(y_test.shape)


In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_seq_len))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))

y_train_int = np.argmax(y_train, axis=1)
y_test_int = np.argmax(y_test, axis=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(x_train, y_train_int, epochs=200, batch_size=32, validation_data=(x_test, y_test_int))



In [ ]:

keras.models.save_model(model,"chatBot.h5")


In [37]:
def generate_response(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_seq_len)
    
    prediction = model.predict(input_seq)
    response_seq = np.argmax(prediction, axis=-1)
    
    response = tokenizer.sequences_to_texts([response_seq])
    return response[0]


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    response = generate_response(user_input)
    print(f"Bot: {response}")
